In [9]:
#Entrainement du modèle à déployer
#1.Importation des librairies
import pandas as pd 
import numpy as np 

import sklearn
### Train Test Split
from sklearn.model_selection import train_test_split
#Gestion unbalanced
import imblearn
from imblearn.over_sampling import SMOTE
#Classifier
import lightgbm
from lightgbm import LGBMClassifier

### Check Accuracy
from sklearn.metrics import accuracy_score

In [14]:
#Importation des données
path = "/Users/adpro/Desktop/Scoring_Model/data_api.csv"
df = pd.read_csv(path)
df = df.set_index("ID")
df.head()

,EXT_SOURCE_2,DAYS_REGISTRATION,EXT_SOURCE_3,DAYS_ID_PUBLISH,DAYS_BIRTH,DAYS_LAST_PHONE_CHANGE,PAYMENT_RATE,REGION_POPULATION_RELATIVE,Target
ID,,,,,,,,,
100002,0.262949,-3648.0,0.139376,-2120,-9461,-1134.0,0.060749,0.018801,1
100003,0.622246,-1186.0,0.535276,-291,-16765,-828.0,0.027598,0.003541,0
100004,0.555912,-4260.0,0.729567,-2531,-19046,-815.0,0.050000,0.010032,0
100006,0.650442,-9833.0,0.535276,-2437,-19005,-617.0,0.094941,0.008019,0
100007,0.322738,-4311.0,0.535276,-3458,-19932,-1106.0,0.042623,0.028663,0


In [58]:
df

,EXT_SOURCE_2,DAYS_REGISTRATION,EXT_SOURCE_3,DAYS_ID_PUBLISH,DAYS_BIRTH,DAYS_LAST_PHONE_CHANGE,PAYMENT_RATE,REGION_POPULATION_RELATIVE,Target
ID,,,,,,,,,
100002,0.262949,-3648.0,0.139376,-2120,-9461,-1134.0,0.060749,0.018801,1
100003,0.622246,-1186.0,0.535276,-291,-16765,-828.0,0.027598,0.003541,0
100004,0.555912,-4260.0,0.729567,-2531,-19046,-815.0,0.050000,0.010032,0
100006,0.650442,-9833.0,0.535276,-2437,-19005,-617.0,0.094941,0.008019,0
100007,0.322738,-4311.0,0.535276,-3458,-19932,-1106.0,0.042623,0.028663,0
...,...,...,...,...,...,...,...,...,...
134821,0.358186,-5054.0,0.722393,-3548,-11143,-247.0,0.035604,0.010147,0
134822,0.579595,-1398.0,0.616122,-4091,-16092,-2225.0,0.029340,0.006629,0
134825,0.524899,-2887.0,0.659406,-959,-8906,-754.0,0.102940,0.025164,0


In [15]:
#pré-proc des données
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

#split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#resampling SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [16]:
#modele
clf = LGBMClassifier(application='binary',
                         is_unbalance=True,
                         metric='auc',
                         max_depth=30,
                         n_estimators=4000,
                         num_leaves=40,
                         subsample=0.1)

In [17]:
#Fiting
clf.fit(X_res,y_res)
#prediction
y_pred=clf.predict(X_test)

In [18]:
score=accuracy_score(y_test,y_pred)
score

0.9055

In [19]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("clf.pkl","wb")
pickle.dump(clf, pickle_out)
pickle_out.close()

In [1]:
#test de requete : 
# import requests
url = "http://127.0.0.1:8000/predict/{customer_id}"
customer_id = 100002

response = requests.get(url.format(customer_id=customer_id))

if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Erreur lors de la requête vers l'API, code de réponse :", response.status_code)
    

{'probabilite': 0.9990583062039323}
